In [178]:
library("tidyr")
library("dplyr")
library("nhs.predict")
library("ggplot2")
library("ggsci")

# input data

In [181]:
input_data <-  read.csv('./input_data_verlification.csv')
id = 3

In [208]:
test5 <- c()
test10 <- c()
test15 <- c()
ids <- c()
for (id in 1:50){
    age.start  <- input_data[id,'age']
    screen     <- input_data[id,'detection']     # Clinically detected = 0, screen detected = 1 (webのsymptoms -> 0)
    size       <- input_data[id,'tumour_size']    # Tumour size mm
    grade      <- input_data[id,'tumour_grade']     # Tumour grade
    nodes      <- input_data[id,'nodes']     # Number positive nodes
    er         <- input_data[id,'er']    # ER+ = 1, ER- = 0
    her2       <- input_data[id,'her2']     # HER2+ = 1, HER2- = 0, missing = 9
    ki67       <- input_data[id,'ki67']     # KI67+ = 1, KI67- = 0, missing = 9
    generation <- input_data[id,'chemo_generation']     # Chemo generation 0, 2 or 3 only
    horm       <- input_data[id,'hormonetherapy']     # Hormone therapy Yes = 1, no = 0
    traz       <- input_data[id,'trastuzumab']     # Trastuzumab therapy Yes = 1, no = 0
    bis        <- input_data[id,'bisphosphonates']
    options    <- input_data[id,'options']
    radio      <- 0     # Radiotherapy Yes = 1, no = 0
    r.enabled  <- 0     # Radiotherapy enabled = 1, disabled = 0

    ##----------------------------------------------------------------
    ##[WINTON FIX] Fix inputs
    screen    <- ifelse(screen == 2, 0.204, screen) #0,1,2(unknown)
    grade     <- ifelse(grade == 9, 2.13, grade) # Tumour grade 1,2,3

    ## ------------------------------------------------------------------------
    time      <- c(1:15)
    age       <- age.start - 1 + time
    ##[WINTON FIX] - Input changed to include grade = 9
    grade.val <- ifelse(er==1, grade, ifelse(grade==2 || grade == 3, 1, 0)) # Grade variable for ER neg

    ## ------------------------------------------------------------------------
    age.mfp.1   <- ifelse(er==1, (age.start/10)^-2-.0287449295, age.start-56.3254902)
    age.beta.1  <- ifelse(er==1, 34.53642, 0.0089827)
    age.mfp.2   <- ifelse(er==1, (age.start/10)^-2*log(age.start/10)-.0510121013, 0)
    age.beta.2  <- ifelse(er==1, -34.20342, 0)
    size.mfp    <- ifelse(er==1, log(size/100)+1.545233938, (size/100)^.5-.5090456276)
    size.beta   <- ifelse(er==1, 0.7530729, 2.093446)
    nodes.mfp   <- ifelse(er==1, log((nodes+1)/10)+1.387566896, log((nodes+1)/10)+1.086916249)
    nodes.beta  <- ifelse(er==1, 0.7060723, .6260541)
    grade.beta  <- ifelse(er==1, 0.746655, 1.129091)
    screen.beta <- ifelse(er==1, -0.22763366, 0)
    her2.beta   <- ifelse(her2==1, 0.2413,
                          ifelse(her2==0, -0.0762,0 ))
    ki67.beta   <- ifelse(ki67==1 & er==1, 0.14904,
                          ifelse(ki67==0 & er==1, -0.11333,0 )) #[WINTON FIX] - Missing 3 at the end

    pi <- age.beta.1*age.mfp.1 + age.beta.2*age.mfp.2 + size.beta*size.mfp +
      nodes.beta*nodes.mfp + grade.beta*grade.val + screen.beta*screen +
      her2.beta + ki67.beta

    ## ----baseline_adjust-----------------------------------------------------
    r.prop   <- 0.69 # Proportion of population receiving radiotherapy
    r.breast <- 0.82 # Relative hazard breast specifi mortality from Darby et al
    r.other  <- 1.07 # Relative hazard other mortality from Darby et al

    if (r.enabled == 1) {
      r.base.br  <- log(1/((1-r.prop) + r.prop*r.breast))
      r.base.oth <- log(1/((1-r.prop) + r.prop*r.other))
    } else {
      r.base.br   <- 0
      r.base.oth  <- 0
    }

    ## ------------------------------------------------------------------------
    # Other mortality prognostic index (mi)
    mi <- 0.0698252*((age.start/10)^2-34.23391957) + r.base.oth

    # Breast cancer mortality prognostic index (pi)
    pi <- age.beta.1*age.mfp.1 + age.beta.2*age.mfp.2 + size.beta*size.mfp +
      nodes.beta*nodes.mfp + grade.beta*grade.val + screen.beta*screen +
      her2.beta + ki67.beta

    c     <- ifelse(generation == 0, 0, ifelse(generation == 2, -.248, -.446))
    h     <- ifelse(horm==1 & er==1, -0.3857, 0)
    h10  <- c(rep(h, 10), rep(-.26+h, 5)) #including both ATLAS and aTTom trials
    t     <- ifelse(her2==1 & traz==1, -.3567, 0)
    b     <- ifelse(bis==1, -0.198, 0) # Only applicable to menopausal women.

    if(r.enabled == 1) {
      r.br  <- ifelse(radio==1, log(r.breast), 0)
      r.oth <- ifelse(radio==1, log(r.other), 0)
    } else {
      r.br = 0
      r.oth = 0
    }
    rx <- tibble(s = rep(0, 15),
                 c = c,
                 h = h,
                 t = t,
                 b = b,
                 hc = h + c,
                 ht = h + t,
                 hb = h + b,
                 ct = c + t, # It is unlikely that hromone therapy would not be offered
                 cb = c + b, # in a woman with ER positive disease
                 tb = t + b,
                 hct = h + c + t,
                 hcb = h + c + b,
                 htb = h + t + b,
                 ctb = c + t + b,
                 hctb = h + c + t + b,
                 h10 = h10,
                 h10c = h10 + c,
                 h10t = h10 + t,
                 h10b = h10 + b,
                 h10ct = h10 + c + t,
                 h10cb = h10 + c + b,
                 h10tb = h10 + t + b,
                 h10ctb = h10 + c + t + b,
                 )

    rx <- rx + pi

    cols <- ncol(rx)

    ## ------------------------------------------------------------------------
    # Generate cumulative baseline other mortality
    base.m.cum.oth <- exp(-6.052919 + (1.079863*log(time)) + (.3255321*time^.5))

    # Generate cumulative survival non-breast mortality
    s.cum.oth <- exp(-exp(mi+r.oth)*base.m.cum.oth)

    # Convert cumulative mortality rate into cumulative risk
    m.cum.oth <- 1- s.cum.oth

    # Annual other mortality rate 年単位の死亡率
    m.oth <- m.cum.oth
    for (i in 2:15) {
      m.oth[i] <- m.cum.oth[i] - m.cum.oth[i-1]
    }


    ## ------------------------------------------------------------------------
    # Generate cumulative baseline breast mortality
    if (er==1) {
      base.m.cum.br <- exp(0.7424402 - 7.527762/time^.5 - 1.812513*log(time)/time^.5)
    } else { base.m.cum.br <- exp(-1.156036 + 0.4707332/time^2 - 3.51355/time)
    }

    # Generate annual baseline breast mortality
    base.m.br <- base.m.cum.br
    for (i in 2:15) {
      base.m.br[i] <- base.m.cum.br[i] - base.m.cum.br[i-1] }

    # Generate the annual breast cancer specific mortality rate
    m.br <- base.m.br*exp(rx)

    # Calculate the cumulative breast cancer mortality rate
    m.cum.br <- apply(m.br, 2, cumsum)

    # Calculate the cumulative breast cancer survival
    s.cum.br <- exp(- m.cum.br)
    m.cum.br <- 1- s.cum.br

    m.br <- m.cum.br
    for (j in 1:cols) {
      for (i in 2:15) {
        m.br[i,j] <- m.cum.br[i,j] - m.cum.br[i-1,j]
      }
    }

    # Cumulative all cause mortality conditional on surviving breast and all cause mortality
    m.cum.all <- 1 - s.cum.oth*s.cum.br
    s.cum.all <- 100-100*m.cum.all

    # Annual all cause mortality
    m.all <- m.cum.all
    for (j in 1:cols) {
      for (i in 2:15) {
        m.all[i,j] <- m.cum.all[i,j] - m.cum.all[i-1,j]
      }
    }

    results_table <- data.frame(s.cum.all)
    select_col <- c('s',options)
    results_selected <- results_table[,select_col]
    #results_selected <- cbind(other, results_selected)
    num_col <- ncol(results_selected)
    results_selected[c(5,10,15),select_col]
    ids <- c(ids, id)
    test5 <- c(test5,results_selected[5,options])
    test10 <- c(test10,results_selected[10,options])
    test15 <- c(test15,results_selected[15,options])
}

In [215]:
df <- data.frame(ids,test5,test10, test15)
write.csv(df, './verlification.csv')